## Pauli Transfer Matrix extraction
$\def\bra#1{\mathinner{\left\langle{#1}\right|}}\def\ket#1{\mathinner{\left|{#1}\right\rangle}}$

In [1]:
import numpy as np; pi = np.pi
import scipy.io as spio
from qutip import *

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# Figures should have a width of a 8.6 cm or 3 3/8 in, the width of a single manuscript column.
golden_mean = (np.sqrt(5)-1.0)/2.0 # Aesthetic ratio
fig_width = 3+3/8 # width in inches
fig_height = fig_width*golden_mean # height in inches (0.75 is the standard height to width ratio in Python)
fig_size = [fig_width, fig_height]
mpl.rcParams.update({
    'axes.labelsize' : 12,
    'font.size' : 12,
    'legend.fontsize' : 8,
    'xtick.labelsize' : 10,
    'ytick.labelsize' : 10,
    'ytick.minor.pad' : -0.5,
    'ytick.minor.pad' : -0.5,
    'ytick.major.size' : 1,
    'ytick.minor.size' : 1,
    'ytick.major.width' : .5,
    'ytick.minor.width' : .5,
    'xtick.major.size' : 1,
    'xtick.minor.size' : 1,
    'xtick.major.width' : .5,
    'xtick.minor.width' : .5,
    'xtick.major.pad' : 1.5,
    'xtick.minor.pad' : 1.5,
    'text.usetex' : True,
    'figure.dpi' : 100,
})

## Parameters

In [26]:
num_lvl = 20 # number of levels
a = destroy(num_lvl) # annihilation operator
K = 1 # kerr amplitude
G = 4*K # two photon pump amplitude
alpha = np.sqrt(G/K) # amplitude

# cat states
cat_plus = (coherent(num_lvl,alpha) + coherent(num_lvl,-alpha)).unit()
cat_minus = (coherent(num_lvl,alpha) - coherent(num_lvl,-alpha)).unit()

# computational basis
up = (cat_plus + cat_minus)/np.sqrt(2)
down = (cat_plus - cat_minus)/np.sqrt(2)

# Identity
I = up*up.dag() + down*down.dag()

# sigma-z in computational basis
sigma_z = up*up.dag() - down*down.dag()

# sigma-x in computational basis
sigma_x = up*down.dag() + down*up.dag()

# sigma-y in computational basis
sigma_y = 1j*(-up*down.dag() + down*up.dag())

# Array with Pauli matrices
P = [I, sigma_x, sigma_y, sigma_z]

## $R_z(\phi)$-gate

In [6]:
# gate time
T_g = 2/K

# initial state
psi0 = (up+down).unit()

# single photon pump amplitude
def E(t,args):
    phi = args['phi']
    return np.pi*phi/(8*T_g*alpha)*np.sin(np.pi*t/T_g)

def rz(phi):
    return (-1j*phi/2*sigma_z).expm()

# Hamiltonian
H0 = - K * pow(a.dag(),2)*pow(a,2) + G * (pow(a.dag(),2) + pow(a,2))
H1 = a.dag() + a
H_tot = [H0,[H1,E]]

# time array
tlist = np.linspace(0,T_g,200)

# angle array
phi_list = np.linspace(-np.pi,np.pi,20)

### quantum map

A quantum map $\mathcal{E}$ is the following.
$$
    \mathcal{E}
    = \begin{pmatrix}
        \mathcal{E}'(\ket{0}\bra{0}) & \cdots & \mathcal{E}'(\ket{0}\bra{20}) \\
        \vdots & \ddots & \vdots \\
        \mathcal{E}'(\ket{20}\bra{0}) & \cdots & \mathcal{E}'(\ket{20}\bra{20})
    \end{pmatrix}
$$
$\mathcal{E}'(\ket{i}\bra{j})$ is $20\times 20$ matrix, so $\mathcal{E}$ is $20^2\times 20^2$ matrix.

In [65]:
gamma = 1/1500 # single-photon loss rate
c_ops = np.sqrt(gamma)*a # collapse operator

# For precise calculation
opt = Options(nsteps=25000, atol=1e-10, rtol=1e-8)

# Quantum map
prop = propagator(H_tot, T_g, c_op_list = [c_ops], args = {'phi': np.pi}, options = opt)

### Kraus map
$$
\mathcal{E}(\rho) = \sum_{k=1}^N A_k\rho A_k^\dagger
$$
where $A_k$ are Kraus-operators

In [23]:
kraus_form = to_kraus(prop)

### Pauli transfer matrix
$$
(R_\Lambda)_{ij} = \frac{1}{d}\sum_{k=1}^N\mathrm{Tr}[P_iA_kP_jA_k^\dagger]
$$

In [82]:
d = 2
# pauli transfer matrix
R = np.zeros((d**2,d**2))
for i in range(d**2):
    for j in range(d**2):
        R[i,j] = 1/d * abs(sum([(P[i]*A_k*P[j]*A_k.dag()).tr() for A_k in kraus_form]))
R = Qobj(R,dims=[[[2],[2]] for i in range(d)]) # Make into a quantum object

In [83]:
ket0 = basis(2,0)
ket1 = basis(2,1)

In [84]:
rho = ket2dm(ket0)

In [85]:
vec_rho = operator_to_vector(rho)